Agregar features segun el nombre de usuario <br>
Usar la prediccion de un predictor como feature para otro predictor <br>
Usar varios clasificadores y promediar los resultados

In [1]:
from collections import Counter

In [2]:
import gc
import numpy as np

In [3]:
gc.collect()

0

In [5]:
from sklearn.metrics import make_scorer

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
from sklearn.ensemble import BaggingClassifier

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import category_encoders

In [7]:
import xgboost as xgb

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


In [9]:
import pandas as pd

In [10]:
predict = pd.read_csv('trocafone_kaggle_test.csv')

In [11]:
a_predecir = pd.read_csv('labels_training_set.csv')

In [65]:
processed = pd.read_csv('new_processed.csv')

In [66]:
processed = processed.set_index('person')
processed = processed.drop('Unnamed: 0',axis=1)
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel,0,1
person,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,14,6.0,Referral,-0.186678,-0.103768
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,0,16.0,Direct,0.414402,-0.442355
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,66,10.0,Paid,0.414402,-0.442355
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,5,20.6,Direct,-0.303714,0.023156
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,2,17.0,Paid,0.668334,0.060729


In [67]:
predict_features = processed.loc[processed.index.isin(predict['person'])].drop(['primer_visita','ultima_visita'],axis=1)

In [68]:
training_features = processed.loc[processed.index.isin(a_predecir['person'])]
training_features['label'] = a_predecir.set_index('person')['label']


training_labels = training_features['label']
training_features = training_features.drop(['primer_visita','ultima_visita','label'],axis=1)

C:\Users\javif\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [69]:
training_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel,0,1
person,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,6,1,14,6.000000,Referral,-0.186678,-0.103768
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,17,1,2,17.000000,Paid,0.668334,0.060729
001802e4,ad campaign hit,iPhone 6S,Smartphone,0,1,4,19,1,0,19.000000,Paid,0.414402,-0.442355
0019e639,viewed product,Motorola Moto G2 3G Dual,Computer,2,19,189,471,115,9,4.095652,Paid,-0.195117,-0.188160
001b0bf9,viewed product,Samsung Galaxy S8 Plus,Ninguno,1,0,2,7,1,9,7.000000,Ninguno,-0.358131,-0.144479


In [70]:
tg = category_encoders.TargetEncoder(smoothing=True)
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [71]:
encoded_training.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.002778,0.042654,0.068258,0,2,0,6,1,14,6.000000,0.066079,-0.186678,-0.103768
1,0.055313,0.050139,0.036301,0,1,3,17,1,2,17.000000,0.042803,0.668334,0.060729
2,0.031056,0.033590,0.036301,0,1,4,19,1,0,19.000000,0.042803,0.414402,-0.442355
3,0.049121,0.030380,0.068258,2,19,189,471,115,9,4.095652,0.042803,-0.195117,-0.188160
4,0.049121,0.060150,0.017361,1,0,2,7,1,9,7.000000,0.017361,-0.358131,-0.144479


In [72]:
forests = RandomForestClassifier()
forests.fit(encoded_training,training_labels)
forests.feature_importances_

array([0.0257361 , 0.08420452, 0.02099191, 0.02901225, 0.06981725,
       0.11270136, 0.12698483, 0.08113473, 0.1318339 , 0.13169952,
       0.03969021, 0.07210185, 0.07409155])

In [73]:
params={
    'n_estimators':[1,2,3,4,5,6,7,8,9,10],
    'max_samples':[1.0],
    'max_features':[1.0], 
}
grid = GridSearchCV(BaggingClassifier(),params,scoring=make_scorer(roc_auc_score),cv=2,verbose=0)

In [74]:
xtr,xte,ytr,yte=train_test_split(encoded_training,training_labels)

In [75]:
grid.fit(xtr,ytr)

GridSearchCV(cv=2, error_score='raise',
       estimator=BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'max_samples': [1.0], 'max_features': [1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(roc_auc_score), verbose=0)

In [76]:
grid.score(X=xte,y=yte)

0.5348910923276984

In [77]:
xgbc= grid.best_estimator_

In [78]:
xgbc.fit(xtr,ytr)
p=xgbc.predict(xte)
roc_auc_score(y_score=p,y_true=yte)

0.5524490680537495

In [ ]:
xgbc.fit(encoded_training,training_labels)

In [ ]:
prediction_submit = xgbc.predict(encoded_predict)

In [ ]:
predict_features['label'] = prediction_submit

In [ ]:
predict_features['label'].value_counts()

In [ ]:
subm = predict.merge(predict_features['label'].reset_index(),how='left',on='person')

In [ ]:
subm = subm.set_index('person')

In [ ]:
subm.to_csv('submit.csv')